In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
from google.colab import drive
drive.mount("/content/gdrive")
%cd gdrive/MyDrive/Ml/nbs/dl2/

from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)


def get_data():
    import os
    import torchvision.datasets as datasets
    root = '../data'
    if not os.path.exists(root):
        os.mkdir(root)
    train_set = datasets.MNIST(root=root, train=True, download=True)
    test_set = datasets.MNIST(root=root, train=False, download=True)
    x_train, x_valid = train_set.data.split([50000, 10000])
    y_train, y_valid = train_set.targets.split([50000, 10000])
    return (x_train.view(50000, -1) / 256.0), y_train.float(), (x_valid.view(10000, -1))/ 256.0, y_valid.float()

x_train,y_train,x_valid,y_valid = get_data()

from exp.nb_03 import *
import torchvision

train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)
nh,bs = 50,64
c = y_train.max().item()+1
loss_func = torch.nn.CrossEntropyLoss()

class DataBunch():
    def __init__(self, train_dl, valid_dl, c=None):
        self.train_dl,self.valid_dl,self.c = train_dl,valid_dl,c
        
        #c is the number of classes
    @property
    def train_ds(self): return self.train_dl.dataset
        
    @property
    def valid_ds(self): return self.valid_dl.dataset

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

def get_model(data, lr=0.5, nh=50):
    m = data.train_ds.x.shape[1] #size of input

    # model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,data.c)) original
    model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,int(data.c)))
    return model, optim.SGD(model.parameters(), lr=lr)

class Learner():
    def __init__(self, model, opt, loss_func, data):
        self.model,self.opt,self.loss_func,self.data = model,opt,loss_func,data

learn = Learner(*get_model(data), loss_func, data)


class Callback():
    def begin_fit(self, learn):
        self.learn = learn
        return True
    
    def after_fit(self): return True
    def begin_epoch(self, epoch):
        self.epoch=epoch
        return True
    def begin_validate(self): return True
    def after_epoch(self): return True
    def begin_batch(self, xb, yb):
        self.xb,self.yb = xb,yb
        return True
    def after_loss(self, loss):
        self.loss = loss
        return True
    def after_backward(self): return True
    def after_step(self): return True



class CallbackHandler():
    # get multiple callbacks which is cbs
    def __init__(self,cbs=None):
        self.cbs = cbs if cbs else []
            
    def begin_fit(self, learn):
        self.learn,self.in_train = learn,True
        learn.stop = False
        res = True
        for cb in self.cbs: 
            res = res and cb.begin_fit(learn)

        return res

    def begin_epoch(self, epoch):
        self.learn.model.train()
        self.in_train=True
        res = True
        for cb in self.cbs: res = res and cb.begin_epoch(epoch)
        return res

    def begin_batch(self, xb, yb):
        res = True
        for cb in self.cbs: res = res and cb.begin_batch(xb, yb)
        return res

    def after_fit(self):
        res = not self.in_train
        for cb in self.cbs: res = res and cb.after_fit()
        return res
    

    def after_loss(self, loss):
        res = self.in_train
        for cb in self.cbs: res = res and cb.after_loss(loss)
        return res

    def after_backward(self):
        res = True
        for cb in self.cbs: res = res and cb.after_backward()
        return res

    def after_step(self):
        res = True
        for cb in self.cbs: res = res and cb.after_step()

        return res

    def do_stop(self):
        try:    
             return self.learn.stop
        finally: 
            self.learn.stop = False

    def begin_validate(self):
        self.learn.model.eval()
        self.in_train=False
        res = True
        for cb in self.cbs: res = res and cb.begin_validate()
        return res

    def after_epoch(self):
        res = True
        for cb in self.cbs: res = res and cb.after_epoch()
        return res


def one_batch(xb, yb, cb):
    if not cb.begin_batch(xb,yb): return

    loss = cb.learn.loss_func(cb.learn.model(xb), yb.long())

    if not cb.after_loss(loss): return
    loss.backward()

    if cb.after_backward(): cb.learn.opt.step()

    if cb.after_step(): cb.learn.opt.zero_grad()

def all_batches(dl, cb):
    for xb,yb in dl:
        one_batch(xb, yb, cb)
        if cb.do_stop(): return

def fit(epochs, learn, cb):
    if not cb.begin_fit(learn): 
        return

    for epoch in range(epochs):

        if not cb.begin_epoch(epoch): 
            continue
        all_batches(learn.data.train_dl, cb)
        
        if cb.begin_validate():
            with torch.no_grad(): 
                all_batches(learn.data.valid_dl, cb)
        if cb.do_stop() or not cb.after_epoch(): 
            break
    cb.after_fit()

Mounted at /content/gdrive
/content/gdrive/MyDrive/Ml/nbs/dl2


In [7]:
class TestCallback(Callback):

    def begin_fit(self, learn):
        super().begin_fit(learn)
        print(f'{"epoch":<20}{"batch":<20}{"trainloss":<40}{"train_error":<40}')
        return True


    def begin_epoch(self,epoch):
        super().begin_epoch(epoch)
        self.n_iters = 0
        print('{:<20}'.format(self.epoch+1),end='')
        return True

    def begin_batch(self, xb, yb):
        super().begin_batch(xb,yb)
        self.tot_loss, self.tot_acc = 0,0 
        return True

    def after_step(self):
        self.n_iters += 1
        pred = self.learn.model(self.xb)
        self.tot_loss = self.learn.loss_func(pred, self.yb.long())
        self.tot_acc = accuracy(pred,self.yb)
        nv = len(self.yb)
        if self.n_iters == 1:
            print('{:<20}{:<40}{:<40}'.format(self.n_iters, (self.tot_loss/nv).item(),(self.tot_acc/nv).item()))
        else:
            print('{:<20}{:<20}{:<40}{:<40}'.format('', self.n_iters, (self.tot_loss/nv).item(),(self.tot_acc/nv).item()))
        
        return True

In [8]:
learn = Learner(*get_model(data), loss_func, data)
fit(3, learn, cb=CallbackHandler([TestCallback()]))

epoch               batch               trainloss                               train_error                             
1                   1                   0.033763401210308075                    0.003662109375                          
                    2                   0.03214311599731445                     0.007080078125                          
                    3                   0.03073994815349579                     0.009765625                             
                    4                   0.029468977823853493                    0.00830078125                           
                    5                   0.026387661695480347                    0.00927734375                           
                    6                   0.025174938142299652                    0.0087890625                            
                    7                   0.02127727121114731                     0.008544921875                          
                    8           